In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
# importing some required libraries/modules
import numpy as np # to deal with arrays
import pandas as pd # to deal with dataframes
import tensorflow as tf # frame work for deep learning
from tensorflow.keras import Sequential,callbacks  
from tensorflow.keras.layers import Flatten,MaxPool2D,Conv2D,Dense  # various layers of CNN


In [4]:
# importing train and test files
df_train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [5]:
# making a copy of original data, we will make changes to copies but not to original data
df_tr=df_train.copy()
df_te=df_test.copy()

In [6]:
#creating folds of data this is not mandatory

from sklearn.model_selection import StratifiedKFold,GridSearchCV
skf=StratifiedKFold(n_splits=5,random_state=0)
for f,(t,v) in enumerate(skf.split(df_tr.iloc[:,1:],df_tr.iloc[:,0])):
    df_tr.loc[v,'Fold']=f

df_tr['Fold']=df_tr['Fold'].astype('int')

In [7]:
# converting the dataframe to numpy array and applying scaling
test = np.array(df_te, dtype=np.float32)/255
#reshaping the data into a form on which we'll train CNN
test = test.reshape(-1,28,28,1)

In [8]:
# Importing mnist dataset, it is a dataset which contains lots of images of hand written digits

from tensorflow.keras.datasets import mnist

(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

In [9]:
#creating a combined dataset of images in our training set and images from mnist dataset

x_mnist = np.concatenate((x_train_mnist,x_test_mnist))
y_mnist = np.concatenate((y_train_mnist,y_test_mnist))

# converting datatype to float and applying scaling
x_mnist = x_mnist.reshape(-1,28,28,1)
x_mnist = x_mnist.astype(np.float32) / 255
y_mnist = tf.keras.utils.to_categorical(y_mnist,num_classes=10)

In [10]:
x_digit = df_tr.drop(['label','Fold'], axis=1)
y_digit = df_tr['label']

In [11]:
# converting the labels into onehot representation
y_digit =tf.keras.utils.to_categorical(y_digit,num_classes=10)
x_digit = np.array(x_digit, dtype=np.float32)/255
x_digit = x_digit.reshape(-1,28,28,1)

In [12]:
X = np.concatenate((x_digit,x_mnist))
y = np.concatenate((y_digit,y_mnist))

In [13]:
# defining the layers of CNN
#Conv2D-- convolution layer
# MaxPool2D-- pooling layer

#Dense-- dense layer

conv1=Conv2D(input_shape=(28,28,1),filters=64,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
conv2=Conv2D(input_shape=(28,28,64),filters=64,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
pool1=MaxPool2D(pool_size=(2,2))


conv3=Conv2D(input_shape=(14,14,64),filters=128,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
conv4=Conv2D(input_shape=(14,14,128),filters=128,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
pool2=MaxPool2D(pool_size=(2,2))

conv5=Conv2D(input_shape=(7,7,128),filters=256,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
conv6=Conv2D(input_shape=(7,7,256),filters=256,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
conv7=Conv2D(input_shape=(14,14,256),filters=256,kernel_size=(3,3),
             activation='relu',padding='same',strides=1)
pool3=MaxPool2D(pool_size=(2,2))





d1=Dense(units=512,activation='relu')
d2=Dense(units=256,activation='relu')
d3=Dense(units=256,activation='relu')



d4=Dense(units=10,activation='softmax')
model=Sequential([conv1,conv2,pool1,conv3,conv4,pool2,pool3
                 ,Flatten(),
                  d1,d2,d3,d4])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator is used for data augmentation

datagen = ImageDataGenerator(rotation_range=90,
                             shear_range=.30,
                             
                  
                  
                  validation_split=0.25,
                  horizontal_flip=False,
                             vertical_flip=False
            )
datagen.fit(X)
generator_train  = datagen.flow(X,y,subset='training',batch_size=2048)
generator_validation  = datagen.flow(X,y,batch_size=2048,subset='validation')

In [15]:
# training the cnn model

early=callbacks.EarlyStopping(patience=30,min_delta=.0001,
                              monitor="val_loss",restore_best_weights=True)
history=model.fit(generator_train,validation_data=generator_validation,callbacks=early,
                 epochs=100)

In [ ]:

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
model.fit(X,y,epochs=60)

In [ ]:
# predicting the results for test set and submitting in required format

y_te=model.predict(test)



predict = np.array(np.round(y_te), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
out = [{'ImageId': i+1, 'Label': predict[i][0]} for i in range(len(predict))]
pd.DataFrame(out).to_csv('submission22.csv', index=False)